# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/matheus/projects/udacity/data_engineering_nanodegree/udacity_data_modeling_with_apache_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkfy 
                    WITH replication = {'class':'SimpleStrategy', 'replication_factor':1};""")

#### Set Keyspace

In [7]:
session.set_keyspace("sparkfy")

In [8]:
tables = ['user_session', 'users_song', 'session']
for t in tables:
    session.execute(f"DROP TABLE IF EXISTS sparkfy.{t}")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
session.execute("""
    CREATE TABLE IF NOT EXISTS sparkfy.session (
        itemInSession int,
        sessionId int,
        artist text, 
        song text,
        length float,
    PRIMARY KEY (itemInSession, sessionId)
    )
""")

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [10]:
session.execute("""
    CREATE TABLE IF NOT EXISTS sparkfy.user_session (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
    PRIMARY KEY((userId, sessionId), itemInSession)
    )
    WITH CLUSTERING ORDER BY (itemInSession ASC);
""")

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [11]:
session.execute(
    """
    CREATE TABLE IF NOT EXISTS sparkfy.users_song(
        song text,
        userId int,
        firstName text,
        lastName text,
    PRIMARY KEY(song, userId)
    )
    """
)

## Query 1

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO sparkfy.session (itemInSession, sessionId, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))

In [13]:
sessionId = 338
itemInSession = 4

query = """
    SELECT 
        artist, song, length
    FROM
        sparkfy.session
    WHERE
        itemInSession={}
        AND sessionId={}
""".format(str(itemInSession), str(sessionId))

rows = session.execute(query)

print("Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4")
for r in rows:
    print(f"""
        Artist: {r.artist}
        Song: {r.song}
        Length(s): {round(r.length, 2)}
    """)

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

        Artist: Faithless
        Song: Music Matters (Mark Knight Dub)
        Length(s): 495.31
    


## Query 2

In [14]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        
        query = "INSERT INTO sparkfy.user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [15]:
userId = 10
sessionId = 182

query = """
    SELECT 
        artist, song, firstName, lastName
    FROM
        sparkfy.user_session
    WHERE
        userId={}
        AND sessionId={}
""".format(str(userId), str(sessionId))

rows = session.execute(query)

print("Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182")
for r in rows:
    print(f"""
        Artist: {r.artist}
        Song: {r.song}
        First Name: {r.firstname}
        Last Name: {r.lastname}
    """)

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

        Artist: Down To The Bone
        Song: Keep On Keepin' On
        First Name: Sylvie
        Last Name: Cruz
    

        Artist: Three Drives
        Song: Greece 2000
        First Name: Sylvie
        Last Name: Cruz
    

        Artist: Sebastien Tellier
        Song: Kilometer
        First Name: Sylvie
        Last Name: Cruz
    

        Artist: Lonnie Gordon
        Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
        First Name: Sylvie
        Last Name: Cruz
    


## Query 3

In [16]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        
        query = "INSERT INTO sparkfy.users_song (song, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
        
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
song = 'All Hands Against His Own'

query = """
    SELECT 
        firstName, lastName
    FROM
        sparkfy.users_song
    WHERE
        song='{}'
""".format(song)

rows = session.execute(query)

print("Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'")
for r in rows:
    print(f"""
        First Name: {r.firstname}
        Last Name: {r.lastname}
    """)

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

        First Name: Jacqueline
        Last Name: Lynch
    

        First Name: Tegan
        Last Name: Levine
    

        First Name: Sara
        Last Name: Johnson
    


### Drop the tables before closing out the sessions

In [18]:
tables = ['user_session', 'users_song', 'session']
for t in tables:
    session.execute(f"DROP TABLE IF EXISTS sparkfy.{t}")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()